In [1]:
import tensorflow as tf

In [2]:
daily_sales_numbers = [21, 22, -108, 31, -1, 32, 34,31]

In [3]:
# creating a tf dataset from list

tf_dataset = tf.data.Dataset.from_tensor_slices(daily_sales_numbers)
tf_dataset

<TensorSliceDataset element_spec=TensorSpec(shape=(), dtype=tf.int32, name=None)>

In [5]:
# itration through tf dataset
for sales in tf_dataset:
    print(sales.numpy())

21
22
-108
31
-1
32
34
31


In [7]:
for sales in tf_dataset.as_numpy_iterator():
    print(sales)

21
22
-108
31
-1
32
34
31


In [9]:
for sales in tf_dataset.take(5):
    print(sales.numpy())

21
22
-108
31
-1


In [10]:
tf_dataset=tf_dataset.filter(lambda x: x>0)

for sales in tf_dataset:
    print(sales.numpy())

21
22
31
32
34
31


In [11]:
# Convert sales numbers from USA dollars ($) to Indian Rupees (INR) Assuming 1->80 conversation rate
tf_dataset = tf_dataset.map(lambda x: x*80)
for sales in tf_dataset.as_numpy_iterator():
    print(sales)

1680
1760
2480
2560
2720
2480


In [13]:
tf_dataset = tf_dataset.shuffle(3)
for sales in tf_dataset.as_numpy_iterator():
    print(sales)

2560
2720
1680
2480
2480
1760


In [14]:
for sales_batch in tf_dataset.batch(2):
    print(sales_batch.numpy())

[1760 2480]
[2560 1680]
[2720 2480]


In [15]:
tf_dataset = tf.data.Dataset.from_tensor_slices(daily_sales_numbers)

tf_dataset = tf_dataset.filter(lambda x: x>0).map(lambda y: y*80).shuffle(2).batch(2)

for sales in tf_dataset.as_numpy_iterator():
    print(sales)

[1760 1680]
[2480 2560]
[2720 2480]


<b style="color:lightgreen"><i>Applying the above operations on Image dataset</i></b>

In [16]:
images = tf.data.Dataset.list_files('./images/*/*', shuffle=False)

In [18]:
img_count = len(images)
img_count

130

In [19]:
type(images)

tensorflow.python.data.ops.dataset_ops.TensorSliceDataset

In [21]:
for img in images.take(5):
    print(img.numpy())

b'.\\images\\cat\\20 Reasons Why Cats Make the Best Pets....jpg'
b'.\\images\\cat\\7 Foods Your Cat Can_t Eat.jpg'
b'.\\images\\cat\\A cat appears to have caught the....jpg'
b'.\\images\\cat\\Adopt-A-Cat Month\xc2\xae - American Humane....jpg'
b'.\\images\\cat\\All About Your Cat_s Tongue.jpg'


In [26]:
images = images.shuffle(130)

for img in images.take(5):
    print(img.numpy())

b'.\\images\\dog\\Hong Kong dog causes panic \xe2\x80\x93 but here_s... (1).jpg'
b'.\\images\\cat\\International Cat Care _ The ultimate....jpg'
b'.\\images\\cat\\The Joys of Owning a Cat - HelpGuide.org.jpg'
b'.\\images\\dog\\Those Puppy Dog Eyes You Can_t Resist....jpg'
b'.\\images\\dog\\How Many Dog Breeds Are There_ _ Hill_s Pet.jpg'


In [27]:
classes = ['Cat', 'Dog']

In [28]:
train_size = int(img_count*0.8)
train_ds = images.take(train_size)
test_ds = images.skip(train_size)

In [29]:
len(train_ds), len(test_ds)

(104, 26)

In [36]:
import os

def get_class_label(path):
    parts = tf.strings.split(path, os.path.sep)
    return parts[-2]

In [42]:
get_class_label(".\\images\\dog\\Hong Kong dog causes panic \xe2\x80\x93 but here_s... (1).jpg")

<tf.Tensor: shape=(), dtype=string, numpy=b'dog'>

In [43]:
def image_processing(path):
    class_label=get_class_label(path)
    
    img = tf.io.read_file(path)
    img = tf.image.decode_jpeg(img)
    img = tf.image.resize(img, [128, 128])
    
    return img, class_label

In [49]:
img, label = image_processing('.\\images\\cat\\International Cat Care _ The ultimate....jpg')
print(img.numpy())
print(label.numpy())

[[[218.03906 218.03906 218.03906]
  [219.5625  219.5625  219.5625 ]
  [219.      219.      219.     ]
  ...
  [226.      231.      235.     ]
  [226.      231.      235.     ]
  [225.      230.      234.     ]]

 [[219.11719 219.11719 219.11719]
  [219.5625  219.5625  219.5625 ]
  [219.      219.      219.     ]
  ...
  [226.      231.      235.     ]
  [226.      231.      235.     ]
  [225.      230.      234.     ]]

 [[219.      219.      219.     ]
  [219.      219.      219.     ]
  [219.      219.      219.     ]
  ...
  [227.      232.      236.     ]
  [226.      231.      235.     ]
  [225.      230.      234.     ]]

 ...

 [[210.      205.      201.     ]
  [210.      205.      201.     ]
  [210.      205.      201.     ]
  ...
  [201.      194.      186.39062]
  [203.80469 195.19531 190.     ]
  [201.8125  196.8125  190.8125 ]]

 [[210.      205.      201.     ]
  [210.      205.      201.     ]
  [210.      205.      201.     ]
  ...
  [201.      194.      188.     ]
  [2

In [50]:
def scaling(image, label):
    return image/255, label

In [52]:
train_ds = train_ds.map(image_processing)
test_ds = test_ds.map(image_processing)

In [53]:
train_ds = train_ds.map(scaling)
test_ds = test_ds.map(scaling)

In [54]:
for image, label in train_ds.take(1):
    print("****",image)
    print("****",label)

**** tf.Tensor(
[[[0.92941177 0.92941177 0.92941177]
  [0.92941177 0.92941177 0.92941177]
  [0.92941177 0.92941177 0.92941177]
  ...
  [0.8012839  0.59736234 0.4130486 ]
  [0.7707903  0.597414   0.42121917]
  [0.81061196 0.63806295 0.46159238]]

 [[0.92941177 0.92941177 0.92941177]
  [0.92941177 0.92941177 0.92941177]
  [0.92941177 0.92941177 0.92941177]
  ...
  [0.7755285  0.5755285  0.3990579 ]
  [0.7993576  0.614613   0.45000193]
  [0.81754464 0.62724704 0.45768994]]

 [[0.92941177 0.92941177 0.92941177]
  [0.92941177 0.92941177 0.92941177]
  [0.92941177 0.92941177 0.92941177]
  ...
  [0.8015108  0.616952   0.45212355]
  [0.8233858  0.62063324 0.46717983]
  [0.785297   0.5815037  0.42427906]]

 ...

 [[0.9998162  0.9998162  0.9998162 ]
  [0.99215686 0.99215686 0.99215686]
  [1.         1.         1.        ]
  ...
  [1.         1.         1.        ]
  [1.         1.         1.        ]
  [1.         1.         1.        ]]

 [[0.9998162  0.9998162  0.9998162 ]
  [0.99215686 0.99215

In [55]:
for image, label in train_ds.take(5):
    print("****Image: ",image.numpy()[0][0])
    print("****Label: ",label.numpy())

****Image:  [0.34335172 0.359038   0.35511643]
****Label:  b'dog'
****Image:  [0.0891391  0.07345282 0.0381587 ]
****Label:  b'dog'
****Image:  [0.85505515 0.85505515 0.85505515]
****Label:  b'cat'
****Image:  [0.14935039 0.19651932 0.14156231]
****Label:  b'cat'
****Image:  [0.49607843 0.4254902  0.38627452]
****Label:  b'cat'
